In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
# ! pip3 install statsmodels
# ! pip3 install termcolor
import pandas as pd
import numpy as np
import pmdarima
#import geopandas as gpdx
import json
import folium as fo
import matplotlib.pyplot as plt
from termcolor import colored
import numpy as np
%matplotlib inline             
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA


print('Hello')

Hello


/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [7]:
os.chdir(r"/home/coffreo/Notebooks/cs-consommation/Extraction")
from fonctions_extraction import * 

In [8]:
coffreo = pd.read_pickle(r"/home/coffreo/Notebooks/cs-consommation/Data/dataframe_filtre.pkl")

In [9]:
coffreo = df_update(coffreo)
coffreo = modif_zip(coffreo)

In [10]:
div_count = dict()
for idx,name in enumerate(coffreo['Division'].value_counts().index.tolist()):
    div_count[name] = coffreo['Division'].value_counts()[idx]


In [11]:
#Création de DataFrame 
X = nb_contrats_actifs(coffreo, '2017-01-01', '2019-01-01', 'Division', 'Entreposage et services auxiliaires des transports')

/home/coffreo/Notebooks/cs-consommation/Extraction/fonctions_extraction.py:489: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_extract = df_extract[["startDate","expectedEndDate"]][df['startDate'].between("2015-01-01", "2021-05-01")]


# Test de stationnarité

In [13]:
#test de stationnarité

from statsmodels.tsa.stattools import adfuller
def check_stationarity(timeseries):
    result = adfuller(timeseries,autolag='AIC')
    dfoutput = pd.Series(result[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    print('The test statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('%s: %.3f' % (key, value))
    if result[1]<0.05:
        return True
    return False


# Fonction calcul d'erreur

In [14]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'minmax':minmax})

# Fonction sarimax_prediction

In [ ]:
def sarimax_prediction(X,horizon,taille_prediction):
    '''
    X : série temporelle
    horizon : horizon de le prédiction pour une itération
    taille_prediction : intervalle de prédiction 
    '''
    
    #plot the decomposition
    decomposition = sm.tsa.seasonal_decompose(X.nbMissions, model='additive')
    plt.rcParams["figure.figsize"] = [16,9]
    decomposition.plot()
    plt.show()

    #seasonality value
    seasonality = int(input("Saisir une valeur pour la saisonnalité de la série temporelle"))
    
    #check the statinnarity
    X_diff = X 
    d_sarima = 0 
    D_sarima = 0

    while not check_stationarity(X_diff):
        X_diff = X_diff.diff()
        X_diff.dropna(inplace=True)
        X_diff = X_diff.diff(seasonality)
        X_diff.dropna(inplace=True)
        d_sarima += 1 
        D_sarima += 1

    #print d and D values
    print('la valeur de d est: ',d_sarima)
    print('la valeur de D est: ',D_sarima)
    
        
    #split the data   
    X.dropna(inplace=True)
    X_array = np.asarray(X)

    train_data = X_array[:len(X_diff)-taille_prediction]
    test_data=X_array[len(X_diff)-taille_prediction:]

    train = pd.DataFrame(data=train_data, index= X[:len(X_diff)-taille_prediction].index , columns=["nbMissions"])
    test = pd.DataFrame(data=test_data, index= X[len(X_diff)-taille_prediction:].index , columns=["nbMissions"])
    
    #plot acf and pacf graphs
    
    plot_acf(X_diff)
    plt.show()
    plt.close()
    plot_pacf(X_diff)
    plt.show()
    plt.close()
    
    #max and min of model's parameters
    p_min = int(input("Saisir une valeur minimale de p "))
    p_max = int(input("Saisir une valeur maximale de p "))
    
    q_min = int(input("Saisir une valeur minimale de q "))
    q_max = int(input("Saisir une valeur maximale de q "))

    
    # choosing the parameters for the model 
    from pmdarima import auto_arima
    stepwise_model = auto_arima(X, start_p=p_min, start_q=q_min,
                               max_p=p_max, max_q=q_max, m=seasonality,
                               start_P=0, seasonal=True,
                               d=d_sarima, D=D_sarima, trace=True,
                               error_action='ignore',  
                               suppress_warnings=True, 
                               stepwise=True)
    print('la valeur de AIC du modèle est de : ',stepwise_model.aic())
    
    
    #Rolling Forecast algorithm 
    start_date = test.index[0]
    l = [start_date]
    while start_date+timedelta(days=horizon) in test.index:
        l.append(start_date)
        start_date = start_date+timedelta(days=horizon)
        
    rolling_predictions = test.copy()
    
    for train_end in l:
        train_data = X[:train_end-timedelta(days=1)]
        train_data.dropna(inplace=True)
        train_data = train_data.astype(np.float)

        model = sm.tsa.statespace.SARIMAX(train_data,order = stepwise_model.order ,seasonal_order = stepwise_model.seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
        model_fit = model.fit()

        pred = model_fit.forecast(horizon, alpha=0.05) 
        date = train_end
        for date in pred.index:
            rolling_predictions[date]  = pred[date]
        date1=date

    train_data = X[:date1-timedelta(days=1)]
    train_data.dropna(inplace=True)
    train_data = train_data.astype(np.float)
    model = sm.tsa.statespace.SARIMAX(train_data,order = stepwise_model.order,seasonal_order = stepwise_model.seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
    model_fit = model.fit()

    pred = model_fit.forecast(len(rolling_predictions)+2-len(rolling_predictions.columns),alpha = 0.05)
    for date in pred.index:
        rolling_predictions[date] = pred[date]

    # Forecast
    fc_sarima = results.forecast(32, alpha=0.05) 
    # Make as pandas series
    fc_series = pd.Series(fc_sarima, index=test.index)
    #lower_series = pd.Series(conf[:, 0], index=test.index)
    #upper_series = pd.Series(conf[:, 1], index=test.index)

    # Plot
    plt.figure(figsize=(12,5), dpi=100)
    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(fc_series, label='forecast')
    #plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8)
    plt.show()

    
    columns = list(rolling_predictions.columns)
    columns.remove(columns[0])
    predictions = dict()
    for column in columns:
        cleanedList = [x for x in rolling_predictions[column] if str(x) != 'nan']
        predictions[str(column)[:10]] = cleanedList[0]
    
    forecast_rolling = pd.DataFrame(predictions.items(), columns=['Date', 'nbMissions'])

    forecast_rolling.drop(columns=['Date'],axis = 1)
    del forecast_rolling['Date']
    fc_rolling = fc_series.copy()
    for i in range(len(fc_series)):
        fc_rolling[i] = forecast_rolling.iloc[i]

    plt.figure(figsize = (16,5))

    # Plot the results 
    plt.figure(figsize=(12,5), dpi=100)
    plt.plot(train, label='training')
    plt.plot(test, label='actual')
    plt.plot(fc_rolling, label='forecast')
    #plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
    plt.title('Forecast vs Actuals')
    plt.legend(loc='upper left', fontsize=8)
    plt.show()

sarimax_prediction(X,15,30)
        